# Summary
- Stage-2 학습 시 Paper에서 제시한 Parameter를 그대로 활용

In [2]:
import yaml
from box import Box

import torch
import torch.nn as nn
import torch.optim as optim

from swin_v2 import SwinTransformerV2

from torch.cuda.amp import autocast, GradScaler
from torch.nn.utils import clip_grad_norm_
from tqdm import tqdm
import time

from timm.data import Mixup
import transformers

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
device = 'cuda:4'
torch.backends.cudnn.benchmark = True

model_save = False
simmim_path = '../../models/swin2/simmim.pth'

In [14]:
model = SwinTransformerV2(pretrained_window_sizes=[6,6,6,6], ape=True, drop_path_rate=0.3)
model.state_dict().keys()

odict_keys(['absolute_pos_embed', 'embeddings.patch_embeddings.weight', 'embeddings.patch_embeddings.bias', 'embeddings.norm.weight', 'embeddings.norm.bias', 'stages.0.blocks.0.attn_mask', 'stages.0.blocks.0.attn.t_scale', 'stages.0.blocks.0.attn.relative_coords_table', 'stages.0.blocks.0.attn.relative_position_index', 'stages.0.blocks.0.attn.crpb_mlp.0.weight', 'stages.0.blocks.0.attn.crpb_mlp.0.bias', 'stages.0.blocks.0.attn.crpb_mlp.3.weight', 'stages.0.blocks.0.attn.qkv.weight', 'stages.0.blocks.0.attn.qkv.bias', 'stages.0.blocks.0.attn.proj.weight', 'stages.0.blocks.0.attn.proj.bias', 'stages.0.blocks.0.norm1.weight', 'stages.0.blocks.0.norm1.bias', 'stages.0.blocks.0.mlp.fc1.weight', 'stages.0.blocks.0.mlp.fc1.bias', 'stages.0.blocks.0.mlp.fc2.weight', 'stages.0.blocks.0.mlp.fc2.bias', 'stages.0.blocks.0.norm2.weight', 'stages.0.blocks.0.norm2.bias', 'stages.0.blocks.1.attn_mask', 'stages.0.blocks.1.attn.t_scale', 'stages.0.blocks.1.attn.relative_coords_table', 'stages.0.blocks.1

In [15]:
model.state_dict()['embeddings.patch_embeddings.weight'][0]

tensor([[[-0.0270, -0.0150, -0.1292, -0.0200],
         [-0.0695,  0.0042,  0.0692,  0.0476],
         [ 0.1091, -0.1437,  0.0946, -0.0676],
         [-0.0275,  0.0488,  0.1311,  0.0886]],

        [[-0.0046,  0.0187, -0.1262, -0.0381],
         [ 0.0380,  0.1383, -0.1189, -0.0180],
         [-0.0150, -0.0903, -0.0621, -0.1111],
         [ 0.1281, -0.1186,  0.0043,  0.0487]],

        [[-0.0366,  0.0098, -0.0726, -0.1226],
         [ 0.0872,  0.0253, -0.1398, -0.0425],
         [ 0.1396,  0.0192,  0.0369,  0.1076],
         [ 0.0522,  0.0779, -0.0197, -0.1169]]])

In [16]:
model.state_dict()['stages.3.blocks.1.attn.crpb_mlp.3.weight'][0]

tensor([-5.7039e-03,  7.9231e-03,  1.6598e-03, -2.4443e-02, -2.0626e-03,
        -1.3681e-02,  1.0584e-02, -2.8501e-02,  1.1764e-02,  5.9102e-02,
         9.1187e-03, -1.2074e-02, -1.5805e-02, -4.2680e-02, -5.3424e-03,
        -2.5962e-02,  2.2780e-02, -2.1294e-02, -2.6561e-02, -8.2797e-03,
         2.1045e-02,  8.8609e-03, -2.0843e-02, -2.9074e-03, -1.2926e-02,
        -2.2739e-03,  1.0008e-02, -3.0646e-02,  4.3791e-02, -6.7375e-04,
        -3.3125e-03, -5.6427e-03, -1.5967e-02, -7.6297e-03,  2.1720e-02,
         1.4745e-03, -9.9285e-03, -7.8507e-03,  2.8790e-02,  9.8445e-03,
         4.7059e-03, -3.0176e-02, -9.3700e-03, -3.0868e-03, -1.0336e-02,
        -3.1913e-02, -1.4835e-02,  3.4004e-02, -4.5951e-03,  1.1914e-02,
         4.2241e-03,  2.3353e-02,  2.8223e-02,  1.3531e-02,  3.1149e-02,
         1.9665e-02, -4.6744e-02, -9.7357e-05,  4.3877e-02, -5.9841e-03,
        -2.7757e-02, -1.3494e-02, -1.7230e-02, -4.4924e-02, -1.8214e-02,
        -3.3901e-02, -3.6728e-02,  4.3705e-02,  7.3

In [17]:
swin_config = yaml.load(open('config/train.yaml'), Loader=yaml.FullLoader)
swin_config

{'MODEL': {'TYPE': 'swinv2',
  'NAME': 'simmim_train',
  'PRETRAINED': '../../models/swin2/simmim.pth',
  'DROP_PATH_RATE': 0.2,
  'SWIN': {'EMBED_DIM': 96,
   'DEPTHS': [2, 2, 6, 2],
   'NUM_HEADS': [3, 6, 12, 24],
   'WINDOW_SIZE': 7,
   'PATCH_SIZE': 4}},
 'DATA': {'IMG_SIZE': 224,
  'MASK_PATCH_SIZE': 32,
  'MASK_RATIO': 0.6,
  'BATCH_SIZE': 960,
  'NUM_WORKERS': 24,
  'DATA_PATH': '../../data/sports'},
 'TRAIN': {'EPOCHS': 20,
  'WARMUP_EPOCHS': 10,
  'BASE_LR': '1e-4',
  'WEIGHT_DECAY': 0.05,
  'CLIP_GRAD': 5}}

In [18]:
def load_pretrained(config, model):
    print(f"==============> Loading weight {config.MODEL.PRETRAINED} for fine-tuning......")
    state_dict = torch.load(config.MODEL.PRETRAINED, map_location='cpu')

    # remain encoder only
    not_encoder_keys = [k for k in state_dict.keys() if 'encoder' not in k]
    for k in not_encoder_keys:
        del state_dict[k]
        
    # remove prefix encoder.
    state_dict = {k.replace('encoder.', ''):v for k, v in state_dict.items()}

    # delete relative_position_index since we always re-init it
    relative_position_index_keys = [k for k in state_dict.keys() if "relative_position_index" in k]
    for k in relative_position_index_keys:
        del state_dict[k]

    # delete relative_coords_table since we always re-init it
    relative_position_index_keys = [k for k in state_dict.keys() if "relative_coords_table" in k]
    for k in relative_position_index_keys:
        del state_dict[k]

    # delete attn_mask since we always re-init it
    attn_mask_keys = [k for k in state_dict.keys() if "attn_mask" in k]
    for k in attn_mask_keys:
        del state_dict[k]

    # bicubic interpolate relative_position_bias_table if not match
    relative_position_bias_table_keys = [k for k in state_dict.keys() if "relative_position_bias_table" in k]
    for k in relative_position_bias_table_keys:
        relative_position_bias_table_pretrained = state_dict[k]
        relative_position_bias_table_current = model.state_dict()[k]
        L1, nH1 = relative_position_bias_table_pretrained.size()
        L2, nH2 = relative_position_bias_table_current.size()
        if nH1 != nH2:
            print(f"Error in loading {k}, passing......")
        else:
            if L1 != L2:
                # bicubic interpolate relative_position_bias_table if not match
                S1 = int(L1 ** 0.5)
                S2 = int(L2 ** 0.5)
                relative_position_bias_table_pretrained_resized = torch.nn.functional.interpolate(
                    relative_position_bias_table_pretrained.permute(1, 0).view(1, nH1, S1, S1), size=(S2, S2),
                    mode='bicubic')
                state_dict[k] = relative_position_bias_table_pretrained_resized.view(nH2, L2).permute(1, 0)

    # bicubic interpolate absolute_pos_embed if not match
    absolute_pos_embed_keys = [k for k in state_dict.keys() if "absolute_pos_embed" in k]
    for k in absolute_pos_embed_keys:
        # dpe
        absolute_pos_embed_pretrained = state_dict[k]
        absolute_pos_embed_current = model.state_dict()[k.replace('encoder.','')]
        _, L1, C1 = absolute_pos_embed_pretrained.size()
        _, L2, C2 = absolute_pos_embed_current.size()
        if C1 != C1:
            print(f"Error in loading {k}, passing......")
        else:
            if L1 != L2:
                S1 = int(L1 ** 0.5)
                S2 = int(L2 ** 0.5)
                absolute_pos_embed_pretrained = absolute_pos_embed_pretrained.reshape(-1, S1, S1, C1)
                absolute_pos_embed_pretrained = absolute_pos_embed_pretrained.permute(0, 3, 1, 2)
                absolute_pos_embed_pretrained_resized = torch.nn.functional.interpolate(
                    absolute_pos_embed_pretrained, size=(S2, S2), mode='bicubic')
                absolute_pos_embed_pretrained_resized = absolute_pos_embed_pretrained_resized.permute(0, 2, 3, 1)
                absolute_pos_embed_pretrained_resized = absolute_pos_embed_pretrained_resized.flatten(1, 2)
                state_dict[k] = absolute_pos_embed_pretrained_resized

    # check classifier, if not match, then re-init classifier to zero
    head_bias_pretrained = state_dict['classifier.bias']
    Nc1 = head_bias_pretrained.shape[0]
    Nc2 = model.classifier.bias.shape[0]
    if (Nc1 != Nc2):
        torch.nn.init.constant_(model.classifier.bias, 0.)
        torch.nn.init.constant_(model.classifier.weight, 0.)
        del state_dict['classifier.weight']
        del state_dict['classifier.bias']
        print(f"Error in loading classifier head, re-init classifier head to 0")

    msg = model.load_state_dict(state_dict, strict=False)
    print(msg)

    print(f"=> loaded successfully '{config.MODEL.PRETRAINED}'")

    torch.cuda.empty_cache()

In [19]:
swin_config = Box(swin_config)
load_pretrained(swin_config, model)

==============> Loading weight ../../models/swin2/simmim.pth for fine-tuning......
_IncompatibleKeys(missing_keys=['stages.0.blocks.0.attn_mask', 'stages.0.blocks.0.attn.relative_coords_table', 'stages.0.blocks.0.attn.relative_position_index', 'stages.0.blocks.1.attn_mask', 'stages.0.blocks.1.attn.relative_coords_table', 'stages.0.blocks.1.attn.relative_position_index', 'stages.1.blocks.0.attn_mask', 'stages.1.blocks.0.attn.relative_coords_table', 'stages.1.blocks.0.attn.relative_position_index', 'stages.1.blocks.1.attn_mask', 'stages.1.blocks.1.attn.relative_coords_table', 'stages.1.blocks.1.attn.relative_position_index', 'stages.2.blocks.0.attn_mask', 'stages.2.blocks.0.attn.relative_coords_table', 'stages.2.blocks.0.attn.relative_position_index', 'stages.2.blocks.1.attn_mask', 'stages.2.blocks.1.attn.relative_coords_table', 'stages.2.blocks.1.attn.relative_position_index', 'stages.2.blocks.2.attn_mask', 'stages.2.blocks.2.attn.relative_coords_table', 'stages.2.blocks.2.attn.relative

In [20]:
model.state_dict()['embeddings.patch_embeddings.weight'][0]

tensor([[[ 0.0301,  0.0009,  0.0505, -0.0364],
         [ 0.1105, -0.0453, -0.0787, -0.0376],
         [-0.0682, -0.0893,  0.0535, -0.0659],
         [-0.0380,  0.0854,  0.0976,  0.0287]],

        [[-0.0771, -0.1077, -0.0139,  0.0576],
         [-0.1156,  0.0058,  0.0420, -0.0442],
         [-0.1327,  0.1288, -0.0719,  0.0688],
         [ 0.0249,  0.1164, -0.0735, -0.0361]],

        [[ 0.0094,  0.1149, -0.0851,  0.1097],
         [ 0.1142, -0.1260,  0.1255, -0.0553],
         [ 0.0574, -0.0123, -0.0088,  0.0854],
         [-0.0981,  0.0671, -0.1177,  0.0822]]])

In [21]:
model.state_dict()['stages.3.blocks.1.attn.crpb_mlp.3.weight'][0]

tensor([ 0.3441,  0.2157, -0.2093,  0.3680, -0.0128, -0.1047,  0.3628,  0.1783,
        -0.1962, -0.0918,  0.2762,  0.3604, -0.1071,  0.3773, -0.0933, -0.0062,
        -0.1436, -0.1425, -0.1067,  0.2658, -0.0970,  0.2234,  0.0165,  0.3350,
        -0.1316, -0.1465,  0.3168, -0.1226, -0.1506, -0.0127,  0.3362, -0.1885,
        -0.1618, -0.1210,  0.3268,  0.2339, -0.1220, -0.0818, -0.0878, -0.1294,
        -0.0691,  0.1114, -0.0910, -0.1057,  0.2985,  0.3047, -0.1025, -0.0852,
        -0.0855,  0.1312, -0.0273, -0.1521,  0.3055,  0.1763, -0.0781,  0.3170,
        -0.0057, -0.0150, -0.0239, -0.1214, -0.0623, -0.2075, -0.0569,  0.2784,
        -0.0833, -0.1258, -0.1253,  0.1136, -0.1609,  0.1525,  0.3013, -0.1182,
         0.0469, -0.1002, -0.1370, -0.1209,  0.2787,  0.0903, -0.1388, -0.1924,
        -0.1242,  0.0175, -0.1391, -0.0981,  0.3561,  0.3985, -0.0734,  0.3098,
         0.0680, -0.1240,  0.2772, -0.0027,  0.2672,  0.3719, -0.1221,  0.2322,
        -0.1118, -0.1924, -0.1657, -0.14

In [22]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [23]:
# Transforms 정의하기
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8,1), interpolation=transforms.InterpolationMode.LANCZOS),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.9, scale=(0.02, 0.33)),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

data_dir = '../../data/sports'
batch_size = 960

train_path = data_dir+'/train'
valid_path = data_dir+'/valid'
test_path = data_dir+'/test'

# dataset load
train_data = ImageFolder(train_path, transform=train_transform)
valid_data = ImageFolder(valid_path, transform=test_transform)
test_data = ImageFolder(test_path, transform=test_transform)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [24]:
max_norm = 5.0 # paper : 100 with G variants

model.to(device)
model_path = '../../models/swin2/model_w_simmim2.pth'

In [25]:
mixup_fn = Mixup(mixup_alpha=.7, 
                cutmix_alpha=.7, 
                prob=.7, 
                switch_prob=0.5, 
                mode='batch',
                label_smoothing=.1,
                num_classes=100)

epochs = 150

criterion = nn.CrossEntropyLoss(label_smoothing=0.)


In [26]:
layer_names = []
for i, (name, params) in enumerate(model.named_parameters()):
    print(f'{i}: {name}')
    layer_names.append(name)

0: absolute_pos_embed
1: embeddings.patch_embeddings.weight
2: embeddings.patch_embeddings.bias
3: embeddings.norm.weight
4: embeddings.norm.bias
5: stages.0.blocks.0.attn.t_scale
6: stages.0.blocks.0.attn.crpb_mlp.0.weight
7: stages.0.blocks.0.attn.crpb_mlp.0.bias
8: stages.0.blocks.0.attn.crpb_mlp.3.weight
9: stages.0.blocks.0.attn.qkv.weight
10: stages.0.blocks.0.attn.qkv.bias
11: stages.0.blocks.0.attn.proj.weight
12: stages.0.blocks.0.attn.proj.bias
13: stages.0.blocks.0.norm1.weight
14: stages.0.blocks.0.norm1.bias
15: stages.0.blocks.0.mlp.fc1.weight
16: stages.0.blocks.0.mlp.fc1.bias
17: stages.0.blocks.0.mlp.fc2.weight
18: stages.0.blocks.0.mlp.fc2.bias
19: stages.0.blocks.0.norm2.weight
20: stages.0.blocks.0.norm2.bias
21: stages.0.blocks.1.attn.t_scale
22: stages.0.blocks.1.attn.crpb_mlp.0.weight
23: stages.0.blocks.1.attn.crpb_mlp.0.bias
24: stages.0.blocks.1.attn.crpb_mlp.3.weight
25: stages.0.blocks.1.attn.qkv.weight
26: stages.0.blocks.1.attn.qkv.bias
27: stages.0.blocks

In [27]:
layer_names.reverse()
layer_names[:5]

['classifier.bias',
 'classifier.weight',
 'layernorm.bias',
 'layernorm.weight',
 'stages.3.blocks.1.norm2.bias']

In [28]:
lr      = 1.4e-3   # paper : 1.4e-3
lr_mult = 0.87     # paper : 0.87
weight_decay = 0.1 # paper : 0.1

param_groups = []
prev_group_name = layer_names[0].split('.')[0]

for idx, name in enumerate(layer_names):
    
    cur_group_name = name.split('.')[0]
    
    if cur_group_name != prev_group_name:
        lr *= lr_mult
    prev_group_name = cur_group_name
    
    print(f"{idx}: {name}'s lr={lr}")
    
    param_groups += [{'params': [ p for n, p in model.named_parameters() if n == name and p.requires_grad],
                      'lr' : lr,
                      'weight_decay': weight_decay}]

0: classifier.bias's lr=0.0014
1: classifier.weight's lr=0.0014
2: layernorm.bias's lr=0.001218
3: layernorm.weight's lr=0.001218
4: stages.3.blocks.1.norm2.bias's lr=0.0010596599999999998
5: stages.3.blocks.1.norm2.weight's lr=0.0010596599999999998
6: stages.3.blocks.1.mlp.fc2.bias's lr=0.0010596599999999998
7: stages.3.blocks.1.mlp.fc2.weight's lr=0.0010596599999999998
8: stages.3.blocks.1.mlp.fc1.bias's lr=0.0010596599999999998
9: stages.3.blocks.1.mlp.fc1.weight's lr=0.0010596599999999998
10: stages.3.blocks.1.norm1.bias's lr=0.0010596599999999998
11: stages.3.blocks.1.norm1.weight's lr=0.0010596599999999998
12: stages.3.blocks.1.attn.proj.bias's lr=0.0010596599999999998
13: stages.3.blocks.1.attn.proj.weight's lr=0.0010596599999999998
14: stages.3.blocks.1.attn.qkv.bias's lr=0.0010596599999999998
15: stages.3.blocks.1.attn.qkv.weight's lr=0.0010596599999999998
16: stages.3.blocks.1.attn.crpb_mlp.3.weight's lr=0.0010596599999999998
17: stages.3.blocks.1.attn.crpb_mlp.0.bias's lr=0.

In [30]:
optimizer = optim.AdamW(param_groups)
warmup_steps = int(len(train_loader)*(epochs)*0.1)
train_steps = len(train_loader)*(epochs)
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer, 
                                                        num_warmup_steps=warmup_steps, 
                                                        num_training_steps=train_steps,
                                                        num_cycles=0.5)

2024-01-17 12:47:45.853747: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-17 12:47:45.853836: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-17 12:47:45.854726: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-17 12:47:45.860065: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-17 12:47:46.538634: W tensorflow/compiler/tf2

In [31]:
training_time = 0
losses = []
val_losses = []
lrs = []
best_loss = float('inf')

# GradScaler 초기화
scaler = GradScaler()

for epoch in range(100):
    model.train()
    start_time = time.time()
    running_loss = 0.0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}")
    
    for _, data in pbar:
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs, labels = mixup_fn(inputs, labels)
        optimizer.zero_grad()

        # AutoCast 적용
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
        # 스케일링된 그라디언트 계산
        scaler.scale(loss).backward()

        # 그라디언트 클리핑 전에 스케일링 제거
        scaler.unscale_(optimizer)
        clip_grad_norm_(model.parameters(), max_norm=max_norm)

        # 옵티마이저 스텝 및 스케일러 업데이트
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
            
        lr = optimizer.param_groups[0]["lr"]
        lrs.append(lr)
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)        

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for data in valid_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
    val_loss /= len(valid_loader)
    val_losses.append(val_loss)
    
    # 모델 저장
    if val_loss < best_loss:
        best_loss = val_loss
        model_save = True
        if model_save:
            torch.save(model.state_dict(), model_path)

    epoch_duration = time.time() - start_time
    training_time += epoch_duration
    
    text = f'\tLoss: {epoch_loss}, Val Loss: {val_loss}, LR: {lr}, Duration: {epoch_duration:.2f} sec'
    
    if model_save:
        text += f' - model saved!'
        model_save = False

    print(text)
        
text = f"Epoch 당 평균 소요시간 : {training_time / epochs:.2f}초"      
print(text)

Epoch 1: 100%|██████████| 15/15 [01:02<00:00,  4.14s/it]


	Loss: 4.618670908610026, Val Loss: 4.50938081741333, LR: 9.333333333333333e-05, Duration: 63.63 sec - model saved!


Epoch 2: 100%|██████████| 15/15 [01:05<00:00,  4.34s/it]


	Loss: 4.475480238596599, Val Loss: 4.227418422698975, LR: 0.00018666666666666666, Duration: 66.46 sec - model saved!


Epoch 3: 100%|██████████| 15/15 [01:00<00:00,  4.04s/it]


...

	Loss: 2.268972651163737, Val Loss: 0.899932861328125, LR: 0.00045294860470029185, Duration: 59.66 sec


Epoch 99: 100%|██████████| 15/15 [00:58<00:00,  3.91s/it]


	Loss: 2.2860596974690757, Val Loss: 0.8425471186637878, LR: 0.0004377753846088615, Duration: 59.63 sec


Epoch 100: 100%|██████████| 15/15 [00:57<00:00,  3.81s/it]


	Loss: 2.3145973682403564, Val Loss: 0.831217885017395, LR: 0.0004227441637725902, Duration: 58.08 sec
Epoch 당 평균 소요시간 : 38.58초


In [32]:
from sklearn.metrics import confusion_matrix
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [33]:
# 예측 수행 및 레이블 저장
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 혼동 행렬 생성
cm = confusion_matrix(all_labels, all_preds)

# 예측과 실제 레이블
y_true = all_labels  # 실제 레이블
y_pred = all_preds  # 모델에 의해 예측된 레이블

# 전체 데이터셋에 대한 정확도
accuracy = accuracy_score(y_true, y_pred)

# 평균 정밀도, 리콜, F1-Score ('weighted')
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

# 판다스 데이터프레임으로 결과 정리
performance_metrics = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Value': [accuracy, precision, recall, f1_score]
})

# 데이터프레임 출력
performance_metrics

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Metric,Value
0,Accuracy,0.834000
1,Precision,0.853119
2,Recall,0.834000
3,F1 Score,0.824243


In [34]:
for epoch in range(10):
    model.train()
    start_time = time.time()
    running_loss = 0.0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}")
    
    for _, data in pbar:
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs, labels = mixup_fn(inputs, labels)
        optimizer.zero_grad()

        # AutoCast 적용
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
        # 스케일링된 그라디언트 계산
        scaler.scale(loss).backward()

        # 그라디언트 클리핑 전에 스케일링 제거
        scaler.unscale_(optimizer)
        clip_grad_norm_(model.parameters(), max_norm=max_norm)

        # 옵티마이저 스텝 및 스케일러 업데이트
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
            
        lr = optimizer.param_groups[0]["lr"]
        lrs.append(lr)
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)        

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for data in valid_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
    val_loss /= len(valid_loader)
    val_losses.append(val_loss)
    
    # 모델 저장
    if val_loss < best_loss:
        best_loss = val_loss
        model_save = True
        if model_save:
            torch.save(model.state_dict(), model_path)

    epoch_duration = time.time() - start_time
    training_time += epoch_duration
    
    text = f'\tLoss: {epoch_loss}, Val Loss: {val_loss}, LR: {lr}, Duration: {epoch_duration:.2f} sec'
    
    if model_save:
        text += f' - model saved!'
        model_save = False

    print(text)
        
text = f"Epoch 당 평균 소요시간 : {training_time / epochs:.2f}초"      
print(text)

# 예측 수행 및 레이블 저장
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 혼동 행렬 생성
cm = confusion_matrix(all_labels, all_preds)

# 예측과 실제 레이블
y_true = all_labels  # 실제 레이블
y_pred = all_preds  # 모델에 의해 예측된 레이블

# 전체 데이터셋에 대한 정확도
accuracy = accuracy_score(y_true, y_pred)

# 평균 정밀도, 리콜, F1-Score ('weighted')
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

# 판다스 데이터프레임으로 결과 정리
performance_metrics = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Value': [accuracy, precision, recall, f1_score]
})

# 데이터프레임 출력
performance_metrics

Epoch 1: 100%|██████████| 15/15 [00:58<00:00,  3.88s/it]


	Loss: 2.315361825625102, Val Loss: 0.8361165523529053, LR: 0.0004078630818627275, Duration: 59.24 sec


Epoch 2: 100%|██████████| 15/15 [00:56<00:00,  3.74s/it]


	Loss: 2.142416755358378, Val Loss: 0.8118091225624084, LR: 0.00039314019724764573, Duration: 57.30 sec - model saved!


Epoch 3: 100%|██████████| 15/15 [01:00<00:00,  4.06s/it]


...

	Loss: 2.361740795771281, Val Loss: 0.7864488363265991, LR: 0.0002951673646824094, Duration: 63.09 sec


Epoch 10: 100%|██████████| 15/15 [00:57<00:00,  3.81s/it]


	Loss: 2.1840132077534995, Val Loss: 0.8022369146347046, LR: 0.0002819889858080498, Duration: 58.38 sec
Epoch 당 평균 소요시간 : 42.61초


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Metric,Value
0,Accuracy,0.840000
1,Precision,0.855460
2,Recall,0.840000
3,F1 Score,0.829451


In [35]:
for epoch in range(10):
    model.train()
    start_time = time.time()
    running_loss = 0.0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}")
    
    for _, data in pbar:
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs, labels = mixup_fn(inputs, labels)
        optimizer.zero_grad()

        # AutoCast 적용
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
        # 스케일링된 그라디언트 계산
        scaler.scale(loss).backward()

        # 그라디언트 클리핑 전에 스케일링 제거
        scaler.unscale_(optimizer)
        clip_grad_norm_(model.parameters(), max_norm=max_norm)

        # 옵티마이저 스텝 및 스케일러 업데이트
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
            
        lr = optimizer.param_groups[0]["lr"]
        lrs.append(lr)
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)        

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for data in valid_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
    val_loss /= len(valid_loader)
    val_losses.append(val_loss)
    
    # 모델 저장
    if val_loss < best_loss:
        best_loss = val_loss
        model_save = True
        if model_save:
            torch.save(model.state_dict(), model_path)

    epoch_duration = time.time() - start_time
    training_time += epoch_duration
    
    text = f'\tLoss: {epoch_loss}, Val Loss: {val_loss}, LR: {lr}, Duration: {epoch_duration:.2f} sec'
    
    if model_save:
        text += f' - model saved!'
        model_save = False

    print(text)
        
text = f"Epoch 당 평균 소요시간 : {training_time / epochs:.2f}초"      
print(text)

# 예측 수행 및 레이블 저장
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 혼동 행렬 생성
cm = confusion_matrix(all_labels, all_preds)

# 예측과 실제 레이블
y_true = all_labels  # 실제 레이블
y_pred = all_preds  # 모델에 의해 예측된 레이블

# 전체 데이터셋에 대한 정확도
accuracy = accuracy_score(y_true, y_pred)

# 평균 정밀도, 리콜, F1-Score ('weighted')
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

# 판다스 데이터프레임으로 결과 정리
performance_metrics = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Value': [accuracy, precision, recall, f1_score]
})

# 데이터프레임 출력
performance_metrics

Epoch 1: 100%|██████████| 15/15 [01:00<00:00,  4.03s/it]


	Loss: 2.2435073614120484, Val Loss: 0.7829024195671082, LR: 0.0002690369672720392, Duration: 61.46 sec


Epoch 2: 100%|██████████| 15/15 [01:02<00:00,  4.17s/it]


	Loss: 2.25953631401062, Val Loss: 0.7946351170539856, LR: 0.0002563183228209027, Duration: 63.61 sec


Epoch 3: 100%|██████████| 15/15 [00:57<00:00,  3.86s/it]


...

Epoch 9: 100%|██████████| 15/15 [00:55<00:00,  3.72s/it]


	Loss: 2.3562647104263306, Val Loss: 0.7664015889167786, LR: 0.00017438397691295056, Duration: 56.80 sec


Epoch 10: 100%|██████████| 15/15 [00:59<00:00,  3.97s/it]


	Loss: 1.9999084313710531, Val Loss: 0.7568098902702332, LR: 0.00016376888981671546, Duration: 60.53 sec
Epoch 당 평균 소요시간 : 46.67초


,Metric,Value
0,Accuracy,0.862000
1,Precision,0.890222
2,Recall,0.862000
3,F1 Score,0.852728


In [36]:
for epoch in range(10):
    model.train()
    start_time = time.time()
    running_loss = 0.0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}")
    
    for _, data in pbar:
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs, labels = mixup_fn(inputs, labels)
        optimizer.zero_grad()

        # AutoCast 적용
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
        # 스케일링된 그라디언트 계산
        scaler.scale(loss).backward()

        # 그라디언트 클리핑 전에 스케일링 제거
        scaler.unscale_(optimizer)
        clip_grad_norm_(model.parameters(), max_norm=max_norm)

        # 옵티마이저 스텝 및 스케일러 업데이트
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
            
        lr = optimizer.param_groups[0]["lr"]
        lrs.append(lr)
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)        

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for data in valid_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
    val_loss /= len(valid_loader)
    val_losses.append(val_loss)
    
    # 모델 저장
    if val_loss < best_loss:
        best_loss = val_loss
        model_save = True
        if model_save:
            torch.save(model.state_dict(), model_path)

    epoch_duration = time.time() - start_time
    training_time += epoch_duration
    
    text = f'\tLoss: {epoch_loss}, Val Loss: {val_loss}, LR: {lr}, Duration: {epoch_duration:.2f} sec'
    
    if model_save:
        text += f' - model saved!'
        model_save = False

    print(text)
        
text = f"Epoch 당 평균 소요시간 : {training_time / epochs:.2f}초"      
print(text)

# 예측 수행 및 레이블 저장
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 혼동 행렬 생성
cm = confusion_matrix(all_labels, all_preds)

# 예측과 실제 레이블
y_true = all_labels  # 실제 레이블
y_pred = all_preds  # 모델에 의해 예측된 레이블

# 전체 데이터셋에 대한 정확도
accuracy = accuracy_score(y_true, y_pred)

# 평균 정밀도, 리콜, F1-Score ('weighted')
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

# 판다스 데이터프레임으로 결과 정리
performance_metrics = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Value': [accuracy, precision, recall, f1_score]
})

# 데이터프레임 출력
performance_metrics

Epoch 1: 100%|██████████| 15/15 [00:59<00:00,  4.00s/it]


	Loss: 2.2095502773920694, Val Loss: 0.7891548871994019, LR: 0.00015344418132633855, Duration: 61.00 sec


Epoch 2: 100%|██████████| 15/15 [01:04<00:00,  4.27s/it]


	Loss: 2.330418340365092, Val Loss: 0.7789267897605896, LR: 0.00014341544245360648, Duration: 65.41 sec


Epoch 3: 100%|██████████| 15/15 [00:57<00:00,  3.86s/it]


...

Epoch 9: 100%|██████████| 15/15 [01:03<00:00,  4.25s/it]


	Loss: 2.092120313644409, Val Loss: 0.7583897709846497, LR: 8.19366849987511e-05, Duration: 64.89 sec


Epoch 10: 100%|██████████| 15/15 [01:04<00:00,  4.29s/it]


	Loss: 1.9113513549168906, Val Loss: 0.7489486932754517, LR: 7.445715177361148e-05, Duration: 65.37 sec
Epoch 당 평균 소요시간 : 50.77초


,Metric,Value
0,Accuracy,0.870000
1,Precision,0.896099
2,Recall,0.870000
3,F1 Score,0.861196


In [37]:
for epoch in range(10):
    model.train()
    start_time = time.time()
    running_loss = 0.0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}")
    
    for _, data in pbar:
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs, labels = mixup_fn(inputs, labels)
        optimizer.zero_grad()

        # AutoCast 적용
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
        # 스케일링된 그라디언트 계산
        scaler.scale(loss).backward()

        # 그라디언트 클리핑 전에 스케일링 제거
        scaler.unscale_(optimizer)
        clip_grad_norm_(model.parameters(), max_norm=max_norm)

        # 옵티마이저 스텝 및 스케일러 업데이트
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
            
        lr = optimizer.param_groups[0]["lr"]
        lrs.append(lr)
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)        

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for data in valid_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
    val_loss /= len(valid_loader)
    val_losses.append(val_loss)
    
    # 모델 저장
    if val_loss < best_loss:
        best_loss = val_loss
        model_save = True
        if model_save:
            torch.save(model.state_dict(), model_path)

    epoch_duration = time.time() - start_time
    training_time += epoch_duration
    
    text = f'\tLoss: {epoch_loss}, Val Loss: {val_loss}, LR: {lr}, Duration: {epoch_duration:.2f} sec'
    
    if model_save:
        text += f' - model saved!'
        model_save = False

    print(text)
        
text = f"Epoch 당 평균 소요시간 : {training_time / epochs:.2f}초"      
print(text)

# 예측 수행 및 레이블 저장
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 혼동 행렬 생성
cm = confusion_matrix(all_labels, all_preds)

# 예측과 실제 레이블
y_true = all_labels  # 실제 레이블
y_pred = all_preds  # 모델에 의해 예측된 레이블

# 전체 데이터셋에 대한 정확도
accuracy = accuracy_score(y_true, y_pred)

# 평균 정밀도, 리콜, F1-Score ('weighted')
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

# 판다스 데이터프레임으로 결과 정리
performance_metrics = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Value': [accuracy, precision, recall, f1_score]
})

# 데이터프레임 출력
performance_metrics

Epoch 1: 100%|██████████| 15/15 [01:01<00:00,  4.08s/it]


	Loss: 2.1929554382960004, Val Loss: 0.760482132434845, LR: 6.731636103603565e-05, Duration: 62.27 sec


Epoch 2: 100%|██████████| 15/15 [01:00<00:00,  4.03s/it]


	Loss: 2.006266164779663, Val Loss: 0.7502516508102417, LR: 6.0518179650179314e-05, Duration: 61.48 sec


Epoch 3: 100%|██████████| 15/15 [00:58<00:00,  3.91s/it]


...

	Loss: 1.9375934680302938, Val Loss: 0.7562308311462402, LR: 2.280934637599534e-05, Duration: 57.25 sec


Epoch 10: 100%|██████████| 15/15 [00:58<00:00,  3.91s/it]


	Loss: 1.926137924194336, Val Loss: 0.7489646673202515, LR: 1.8868590594123336e-05, Duration: 59.65 sec
Epoch 당 평균 소요시간 : 54.80초


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Metric,Value
0,Accuracy,0.854000
1,Precision,0.869440
2,Recall,0.854000
3,F1 Score,0.840944


In [38]:
for epoch in range(10):
    model.train()
    start_time = time.time()
    running_loss = 0.0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}")
    
    for _, data in pbar:
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs, labels = mixup_fn(inputs, labels)
        optimizer.zero_grad()

        # AutoCast 적용
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
        # 스케일링된 그라디언트 계산
        scaler.scale(loss).backward()

        # 그라디언트 클리핑 전에 스케일링 제거
        scaler.unscale_(optimizer)
        clip_grad_norm_(model.parameters(), max_norm=max_norm)

        # 옵티마이저 스텝 및 스케일러 업데이트
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
            
        lr = optimizer.param_groups[0]["lr"]
        lrs.append(lr)
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)        

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for data in valid_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
    val_loss /= len(valid_loader)
    val_losses.append(val_loss)
    
    # 모델 저장
    if val_loss < best_loss:
        best_loss = val_loss
        model_save = True
        if model_save:
            torch.save(model.state_dict(), model_path)

    epoch_duration = time.time() - start_time
    training_time += epoch_duration
    
    text = f'\tLoss: {epoch_loss}, Val Loss: {val_loss}, LR: {lr}, Duration: {epoch_duration:.2f} sec'
    
    if model_save:
        text += f' - model saved!'
        model_save = False

    print(text)
        
text = f"Epoch 당 평균 소요시간 : {training_time / epochs:.2f}초"      
print(text)

# 예측 수행 및 레이블 저장
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 혼동 행렬 생성
cm = confusion_matrix(all_labels, all_preds)

# 예측과 실제 레이블
y_true = all_labels  # 실제 레이블
y_pred = all_preds  # 모델에 의해 예측된 레이블

# 전체 데이터셋에 대한 정확도
accuracy = accuracy_score(y_true, y_pred)

# 평균 정밀도, 리콜, F1-Score ('weighted')
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

# 판다스 데이터프레임으로 결과 정리
performance_metrics = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Value': [accuracy, precision, recall, f1_score]
})

# 데이터프레임 출력
performance_metrics

Epoch 1: 100%|██████████| 15/15 [00:59<00:00,  3.94s/it]


	Loss: 2.2802740414937337, Val Loss: 0.7501404285430908, LR: 1.5296679486336016e-05, Duration: 60.10 sec


Epoch 2: 100%|██████████| 15/15 [01:02<00:00,  4.14s/it]


	Loss: 1.7234619537989297, Val Loss: 0.7468369007110596, LR: 1.2095547305510733e-05, Duration: 63.02 sec


Epoch 3: 100%|██████████| 15/15 [01:00<00:00,  4.02s/it]


...

	Loss: 2.0547617197036745, Val Loss: 0.7437141537666321, LR: 1.8953117437680023e-07, Duration: 64.09 sec


Epoch 10: 100%|██████████| 15/15 [01:01<00:00,  4.11s/it]


	Loss: 2.0507237116495767, Val Loss: 0.7436926960945129, LR: 0.0, Duration: 62.97 sec
Epoch 당 평균 소요시간 : 58.93초


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Metric,Value
0,Accuracy,0.862000
1,Precision,0.876028
2,Recall,0.862000
3,F1 Score,0.849429
